In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
regex_pattern = r'\\b\\d{10}\\b'

In [0]:
spark.sql(f"""
create or replace table {catalog_name}.udp_wcm_gold_vcm_dwh.f_transaction_loyalty 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
WITH winplus_phone AS (
  SELECT order_code, customer_phone
  from {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_orders
  WHERE channel = 'WINPLUS' AND customer_phone IS NOT NULL
  GROUP BY 1, 2
),
f_sale_prod AS (
  SELECT transaction_no, calday, 
      transaction_type,
      SUM(sale_amount) sale_amount,
      SUM(rev_amt_pos) rev_amt_pos
  FROM (
      SELECT LTRIM('0' ,transaction_no) transaction_no,
        calday,
        product_id,
        transaction_type,
        SUM(sale_amount) sale_amount,
        SUM(sale_amount - tax_amount + IF(discount_type IN ('ZRTA', 'ZCRT'), discount_value, 0)) rev_amt_pos
      FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod
      WHERE calday >= '2022-07-01'
        AND product_id IS NOT NULL
      GROUP BY 1, 2, 3, 4
  ) A
  left join {catalog_name}.udp_wcm_gold_vcm_dwh.d_product product on a.product_id = product.product_id
  WHERE product.mch2_id IN ('101', '102','201', '202', '203')
  GROUP BY 1, 2, 3
),
list_transaction AS (
  SELECT DISTINCT 'WINPLUS' source,
    A.calday,
    LTRIM('0', receipt_no) transaction_no,
    mem.capilary_customer_id ordercustomerid,
    B.customer_phone mobile_number,
    A.store_id
  FROM {catalog_name}.udp_wcm_gold_fin_winplus_ss.winplus_order_bill_header_vw A
  LEFT JOIN winplus_phone B ON A.receipt_no = B.order_code
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_win_membership mem ON {catalog_name}.default.fn_clean_mobile_no(B.customer_phone) = mem.mobile_number
  WHERE A.member_type IS NOT NULL
    AND A.member_type <> '0'
    AND A.customer_name <> 'UNKNOW'
  UNION ALL
  SELECT
    'POS' source,
    A.calday,
    LTRIM('0', receipt_no) transaction_no,
    B.capilary_customer_id customer_id,
    A.mobile_number,
    A.store_id
  FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_saleout_bill_header A
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_win_membership B ON A.mobile_number = B.mobile_number
  WHERE A.mobile_number IS NOT NULL
    AND LENGTH(A.mobile_number) > 0
    AND IF(
  {catalog_name}.default.fn_clean_mobile_no(A.mobile_number) RLIKE '{regex_pattern}',
  1,
  0
) = 1
  UNION ALL
  SELECT *
  FROM (
    WITH online_trans AS (
      SELECT calday, receipt_no, ref_03_origin_trx_num
      FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_saleout_bill_header
      WHERE ref_03_origin_trx_num LIKE '17%'
      GROUP BY 1, 2, 3
    ),
    lyt_orders AS (
      SELECT LTRIM('0', order_id) orderid,
          create_date createdate,
          order_customer_id ordercustomerid,
          sale_store_id salestoreid
      FROM {catalog_name}.udp_wcm_silver_cx_loyalty.lyt_orders
      WHERE CASE
        WHEN TO_DATE(create_date,'yyyy-MM-dd HH:mm:ss.SSS') IS NOT NULL
        THEN TO_DATE(create_date,'yyyy-MM-dd HH:mm:ss.SSS')
        ELSE TO_DATE(create_date,'MM/dd/yyyy HH:mm:ss')
      END >= '2022-07-01'
        AND order_customer_id IS NOT NULL
        AND LENGTH(order_customer_id) > 0
      QUALIFY ROW_NUMBER() OVER (PARTITION BY orderid
      ORDER BY CASE
             WHEN TO_DATE(create_date,'yyyy-MM-dd HH:mm:ss.SSS') IS NOT NULL
             THEN TO_DATE(create_date,'yyyy-MM-dd HH:mm:ss.SSS')
             ELSE TO_DATE(create_date,'MM/dd/yyyy HH:mm:ss')
           END DESC) = 1
    )
    SELECT DISTINCT
      'ONLINE' source,
      CASE
        WHEN TO_DATE(createdate,'yyyy-MM-dd HH:mm:ss.SSS') IS NOT NULL
        THEN TO_DATE(createdate,'yyyy-MM-dd HH:mm:ss.SSS')
        ELSE TO_DATE(createdate,'MM/dd/yyyy HH:mm:ss')
      END AS calday,
      CASE
          WHEN online_trans.receipt_no IS NOT NULL THEN online_trans.receipt_no
          ELSE orderid
      END transaction_no,
      CASE
          WHEN B.customer_id IS NOT NULL THEN B.capilary_customer_id
          WHEN C.capilary_customer_id IS NOT NULL THEN C.capilary_customer_id
      END customer_id,
      CASE
          WHEN B.customer_id IS NOT NULL THEN B.mobile_number
          WHEN C.capilary_customer_id IS NOT NULL THEN C.mobile_number
      END mobile_number,
      A.salestoreid store_id
    FROM lyt_orders A
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_win_membership B ON A.ordercustomerid = B.customer_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_win_membership C ON A.ordercustomerid = C.capilary_customer_id
    LEFT JOIN online_trans ON A.orderid = online_trans.ref_03_origin_trx_num
    )
)
SELECT A.*, B.transaction_type, B.sale_amount, B.rev_amt_pos
FROM list_transaction A
LEFT JOIN f_sale_prod B USING(transaction_no, calday)
""")